In [4]:
#1. Data Collection on the preprocessed file
import pandas as pd
dataset1 = pd.read_csv("preprocessed_LoadBalancer.csv", index_col= None)

In [5]:
print(dataset1.head())
print(dataset1.columns)

  Server Status Data Progress  Network Traffic (MB/s)  Request Size (MB)  \
0            s2    inprogress                   86.55              44.09   
1            s3    inprogress                   37.39             346.07   
2            s6      complete                   33.81             109.68   
3            s5    inprogress                   45.27             283.95   
4            s4      complete                   74.16              99.97   

   Threshold Server Load  Response Time (ms)  Priority (0/1)  
0       0.77      medium                 359               1  
1       0.95         low                 200               0  
2       0.78         low                 447               1  
3       0.92         low                 408               1  
4       0.91      medium                 320               1  
Index(['Server Status', 'Data Progress', 'Network Traffic (MB/s)',
       'Request Size (MB)', 'Threshold', 'Server Load', 'Response Time (ms)',
       'Priority (0/

In [6]:
# preprocessing by converting categorial data to numerical
df2=pd.get_dummies(dataset1, drop_first=True)


In [15]:
df2

,Network Traffic (MB/s),Request Size (MB),Threshold,Response Time (ms),Priority (0/1),Server Status_s2,Server Status_s3,Server Status_s4,Server Status_s5,Server Status_s6,Server Status_s7,Data Progress_inprogress,Server Load_low,Server Load_medium
0,86.55,44.09,0.77,359,1,True,False,False,False,False,False,True,False,True
1,37.39,346.07,0.95,200,0,False,True,False,False,False,False,True,True,False
2,33.81,109.68,0.78,447,1,False,False,False,False,True,False,False,True,False
3,45.27,283.95,0.92,408,1,False,False,False,True,False,False,True,True,False
4,74.16,99.97,0.91,320,1,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1249,300.00,128.00,8.00,172,1,False,False,False,False,False,True,True,False,True
1250,569.00,411.00,3.00,857,0,False,False,True,False,False,False,False,False,False
1251,703.00,79.00,3.00,635,1,False,False,False,False,False,True,True,False,False
1252,147.00,562.00,9.00,823,0,False,False,True,False,False,False,True,False,False


In [7]:
indep_X = df2.drop('Priority (0/1)', axis=1)


In [19]:
indep_X

,Network Traffic (MB/s),Request Size (MB),Threshold,Response Time (ms),Server Status_s2,Server Status_s3,Server Status_s4,Server Status_s5,Server Status_s6,Server Status_s7,Data Progress_inprogress,Server Load_low,Server Load_medium
0,86.55,44.09,0.77,359,True,False,False,False,False,False,True,False,True
1,37.39,346.07,0.95,200,False,True,False,False,False,False,True,True,False
2,33.81,109.68,0.78,447,False,False,False,False,True,False,False,True,False
3,45.27,283.95,0.92,408,False,False,False,True,False,False,True,True,False
4,74.16,99.97,0.91,320,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1249,300.00,128.00,8.00,172,False,False,False,False,False,True,True,False,True
1250,569.00,411.00,3.00,857,False,False,True,False,False,False,False,False,False
1251,703.00,79.00,3.00,635,False,False,False,False,False,True,True,False,False
1252,147.00,562.00,9.00,823,False,False,True,False,False,False,True,False,False


In [8]:
dep_Y = df2['Priority (0/1)']

In [9]:

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(indep_X, dep_Y, test_size =0.25, random_state= 0)
      


In [10]:
#standardize the input
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import numpy as np 

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

param_grid = { 'criterion': [ "absolute_error", "mae" ,"mse", "mae", "friedman_mse", "absolute_error", "poisson"], 
              'splitter' : ["best", "random"],
              'max_features' : [None, "auto", "sqrt", "log2"]
             }

grid = GridSearchCV(DecisionTreeRegressor(), param_grid, refit = True, verbose = 3, n_jobs = 2)

#create the model for all the Grid combinations
grid.fit(X_train, Y_train)

print("Printing before tuning: ", grid.best_params_, " best score: ", grid.best_score_)

y_grid_predictions = grid.predict(X_test)

from sklearn.metrics import r2_score
r_score = r2_score(Y_test, y_grid_predictions)

print("The R score value for best param {}: ".format(grid.best_params_), r_score)

Fitting 5 folds for each of 56 candidates, totalling 280 fits
Printing before tuning:  {'criterion': 'poisson', 'max_features': 'sqrt', 'splitter': 'random'}  best score:  -0.3512746540649438
The R score value for best param {'criterion': 'poisson', 'max_features': 'sqrt', 'splitter': 'random'}:  -0.2755292370069842


In [17]:
# 4 A 5) Feature Selection: Grid SVM
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

param_grid = {'kernel': [ #'poly', 'linear', 
                         'poly'], #, 'sigmoid'],
              'C': [1],#, 10, 100],# 1000, 3000],
              'gamma': [1], #, 'scale', 'auto'],
              'degree': [2],
              'epsilon' :  [0.001]
             }

grid = GridSearchCV(SVR(), param_grid, refit = True, verbose = 3, n_jobs = 2)

#create the model for all the Grid combinations
grid.fit(X_train, Y_train)

print("Printing before tuning: ", grid.best_params_, " best score: ", grid.best_score_)

y_grid_predictions = grid.predict(X_test)

from sklearn.metrics import r2_score
r_score = r2_score(Y_test, y_grid_predictions)

print("The R score value for best param {}: ".format(grid.best_params_), r_score)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Printing before tuning:  {'C': 1, 'degree': 2, 'epsilon': 0.001, 'gamma': 1, 'kernel': 'poly'}  best score:  -0.5874656967693681
The R score value for best param {'C': 1, 'degree': 2, 'epsilon': 0.001, 'gamma': 1, 'kernel': 'poly'}:  -0.6966794890246772


In [32]:
# 4 A 5) Feature Selection: Grid RF
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid= { 'criterion': [ "absolute_error", "squared_error", "friedman_mse", "mse", "mae"],
             'max_features' : [None, "auto", "sqrt", "log2"],
             'n_estimators' : [10, 100, 200],
             'oob_score' : [True, False],
            'random_state' :  [30, 42, 50]}

grid = GridSearchCV(RandomForestRegressor(), param_grid, refit=True, verbose=3, n_jobs= -1)
grid.fit(X_train, Y_train)

y_grid_prediction = grid.predict(X_test)
# evalulate metrics R2 score
from sklearn.metrics import r2_score
r_score = r2_score(Y_test, y_grid_prediction)

print("The R Score value for the best parameter {}: ".format(grid.best_params_), r_score)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits
The R Score value for the best parameter {'criterion': 'squared_error', 'max_features': 'sqrt', 'n_estimators': 200, 'oob_score': True, 'random_state': 50}:  -0.05787760825163413


In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid_lr = {'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs'],
    'penalty': ['l2']
                }
classifier_grid_lr = GridSearchCV(LogisticRegression(),param_grid_lr, refit = True, verbose= 3, n_jobs= -1)
classifier_grid_lr.fit(X_train, Y_train)

print("Printing Logistic Regression best param after tuning: classifier_grid_lr.best_params_ ", classifier_grid_lr.best_params_)

y_grid_pred_lr = classifier_grid_lr.predict(X_test)

print ("Y_test", len(Y_test))
print ("y_grid_pred_lr",len( y_grid_pred_lr))
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_grid_pred_lr)

print("cm LR:" , cm)
from sklearn.metrics import classification_report
report = classification_report(Y_test,y_grid_pred_lr)

print(" Logical Regression report:", report)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Printing Logistic Regression best param after tuning: classifier_grid_lr.best_params_  {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
Y_test 314
y_grid_pred_lr 314
cm LR: [[82 88]
 [71 73]]
 Logical Regression report:               precision    recall  f1-score   support

           0       0.54      0.48      0.51       170
           1       0.45      0.51      0.48       144

    accuracy                           0.49       314
   macro avg       0.49      0.49      0.49       314
weighted avg       0.50      0.49      0.49       314



In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
param_grid_dt = {
    'criterion': ['gini', 'entropy'],  # Splitting criteria
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split
    'min_samples_leaf': [1, 2, 5],    # Minimum number of samples per leaf node
    'max_features': [None, 'sqrt', 'log2'],  # Number of features to consider for the best split
    'splitter': ['best', 'random']   # Strategy to split at each node
}
classifier_grid_dt = GridSearchCV(DecisionTreeClassifier(),param_grid_dt, refit = True, verbose= 3, n_jobs= -1)
classifier_grid_dt.fit(X_train, Y_train)

y_grid_pred_dt = classifier_grid_dt.predict(X_test)

print ("Y_test", len(Y_test))
print ("y_grid_pred_dt",len( y_grid_pred_dt))
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_grid_pred_dt)

print("cm DT:" , cm)
from sklearn.metrics import classification_report
report = classification_report(Y_test,y_grid_pred_dt)

print(" Decision Tree report:", report)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Y_test 314
y_grid_pred_dt 314
cm DT: [[120  50]
 [102  42]]
 Decision Tree report:               precision    recall  f1-score   support

           0       0.54      0.71      0.61       170
           1       0.46      0.29      0.36       144

    accuracy                           0.52       314
   macro avg       0.50      0.50      0.48       314
weighted avg       0.50      0.52      0.49       314



In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
param_grid_rf = {
    'n_estimators': [50, 100, 200],       # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],     # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],     # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4],       # Minimum number of samples required to be a leaf node
    'max_features': ['sqrt', 'log2'],    # Number of features to consider when looking for the best split
    'bootstrap': [True, False]           # Whether bootstrap samples are used when building trees
}
 

classifier_grid_rf = GridSearchCV(RandomForestClassifier(),param_grid_rf, refit = True, verbose= 3, n_jobs= -1)
classifier_grid_rf.fit(X_train, Y_train)

y_grid_pred_rf = classifier_grid_rf.predict(X_test)

print ("Y_test", len(Y_test))
print ("y_grid_pred_rf",len( y_grid_pred_rf))
from sklearn.metrics import confusion_matrix
cm_rf = confusion_matrix(Y_test, y_grid_pred_rf)

print("cm RF:" , cm_rf)
from sklearn.metrics import classification_report
report = classification_report(Y_test,y_grid_pred_rf)

print(" Random Forest report:", report)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Y_test 314
y_grid_pred_rf 314
cm RF: [[88 82]
 [72 72]]
 Random Forest report:               precision    recall  f1-score   support

           0       0.55      0.52      0.53       170
           1       0.47      0.50      0.48       144

    accuracy                           0.51       314
   macro avg       0.51      0.51      0.51       314
weighted avg       0.51      0.51      0.51       314



In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC 
param_grid_svc = {
    'C': [0.1, 1, 10, 100],        # Regularization parameter
    'kernel': ['poly'], #'linear', 'poly', 'rbf', 'sigmoid'],  # Kernel types
    'degree': [2] , #, 3, 4],           # Degree of the polynomial kernel (used if kernel='poly')
    'gamma': ['scale'], #', 'auto', 0.1, 1, 10],  # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
    'class_weight': [None, 'balanced'],  # Weighing classes (useful for imbalanced datasets)
    'shrinking': [True, False]    # Whether to use shrinking heuristic
}
 

classifier_grid_svc = GridSearchCV(SVC(),param_grid_svc, refit = True, verbose= 3, n_jobs= -1)
classifier_grid_svc.fit(X_train, Y_train)

y_grid_pred_svc = classifier_grid_svc.predict(X_test)

print ("Y_test", len(Y_test))
print ("y_grid_pred_svc",len( y_grid_pred_svc))
from sklearn.metrics import confusion_matrix
cm_svc = confusion_matrix(Y_test, y_grid_pred_svc)

print("cm SVC:" , cm_svc)
from sklearn.metrics import classification_report
report = classification_report(Y_test,y_grid_pred_svc)

print(" SVCt report:", report)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Y_test 314
y_grid_pred_svc 314
cm SVC: [[75 95]
 [68 76]]
 SVCt report:               precision    recall  f1-score   support

           0       0.52      0.44      0.48       170
           1       0.44      0.53      0.48       144

    accuracy                           0.48       314
   macro avg       0.48      0.48      0.48       314
weighted avg       0.49      0.48      0.48       314



In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
param_grid_gnb = { 
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]  # Variance smoothing values
}
 
classifier_grid_gnb = GridSearchCV(GaussianNB(),param_grid_gnb, cv=5)
classifier_grid_gnb.fit(X_train, Y_train)

y_grid_pred_gnb = classifier_grid_gnb.predict(X_test)

print ("Y_test", len(Y_test))
print ("y_grid_pred_gnb",len( y_grid_pred_gnb))
from sklearn.metrics import confusion_matrix
cm_gnb = confusion_matrix(Y_test, y_grid_pred_gnb)

print("cm gnb:" , cm_gnb)
from sklearn.metrics import classification_report
report = classification_report(Y_test,y_grid_pred_gnb)

print(" Naive Bayes Gaussian BM report:", report)

Y_test 314
y_grid_pred_gnb 314
cm gnb: [[96 74]
 [74 70]]
 Naive Bayes Gaussian BM report:               precision    recall  f1-score   support

           0       0.56      0.56      0.56       170
           1       0.49      0.49      0.49       144

    accuracy                           0.53       314
   macro avg       0.53      0.53      0.53       314
weighted avg       0.53      0.53      0.53       314



In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier 
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],  # Number of neighbors to consider
    'weights': ['uniform', 'distance'],  # Uniform or distance-based weighting
    'metric': ['euclidean', 'manhattan', 'minkowski']  # Distance metric
}
 
classifier_grid_knn = GridSearchCV(KNeighborsClassifier(),param_grid_knn, refit = True, verbose= 3, n_jobs= -1)
classifier_grid_knn.fit(X_train, Y_train)

y_grid_pred_knn = classifier_grid_knn.predict(X_test)

print ("Y_test", len(Y_test))
print ("y_grid_pred_knn",len( y_grid_pred_knn))
from sklearn.metrics import confusion_matrix
cm_knn = confusion_matrix(Y_test, y_grid_pred_knn)

print("cm knn:" , cm_knn)
from sklearn.metrics import classification_report
report = classification_report(Y_test,y_grid_pred_knn)

print("K Nearest neighbor report:", report)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Y_test 314
y_grid_pred_knn 314
cm knn: [[80 90]
 [72 72]]
K Nearest neighbor report:               precision    recall  f1-score   support

           0       0.53      0.47      0.50       170
           1       0.44      0.50      0.47       144

    accuracy                           0.48       314
   macro avg       0.49      0.49      0.48       314
weighted avg       0.49      0.48      0.48       314

